# Домашнее задание
С вашим датасетом нужно проделать примерно то же самое, что мы делали с примером "12-6. Оценка стоимости квартиры.ipynb" Обратите внимание, что у вас - может быть задача классификации, а мы решали задачу регрессии  (также можно посмотреть 19-02. Adult.ipynb, где решается задача классификации) (https://github.com/NikolaiZolotykh/MachineLearningCourse). Все должно быть выложено на ваш гит.

А именно:
0. Описать задачу словами
1. Прочитать данные
2. Визуализировать данные и вычислить основные характеристики (среднее, разброс, корреляционную матрицу и т.д.). Интерпретировать.
3. Обработать пропущенные значения (или убедиться, что их нет)
4. Обработать категориальные признаки
5. Провести нормализацию (или объяснить, почему в вашем случае она не нужна)
6. Разбить данные на обучающую и тестовую выборки
7. Запустить классификатор (регрессию) ближайших соседей или другой (аргументировать свой выбор)
8. Подобрать оптимальное значение к-ва ближайших соседей (или другого релевантного гиперпараметра)
9. Вычислить ошибки на обучающей и тестовой выборках. Сделать выводы 
10. Запустить другие классификаторы и выбать параметры. Выбрать релевантные! Сравнить результаты
11. как-то побороться с несбалансированностью классов (если она есть)
12. исключить коррелированные переменные (объяснить зачем)
13. Сделать общие выводы

## 0. Описание задачи
Задача: Классификация различных типов ритмов сердца (нозологий) на основе набора данных, содержащего
параметры, описывающие работу сердечно-сосудистой системы. Цель — создать модель, которая сможет 
правильно классифицировать различные состояния ритма сердца.

Набор данных:
- Файл three_bases_meta.csv содержит метаданные с описанием признаков.
- Файлы three_bases_X.npy и three_bases_Y.npy содержат признаки и целевые классы для задачи.

## 1. Импорт библиотек и загрузка данных